In [12]:
##

## Similar to item_CF_numba, but read different source, and should be code freeze once validated.

In [1]:
import os
import gc
import heapq
import pickle
import numba as nb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import math

In [2]:
%%time
df = pd.read_csv("../../allData/submission_phase_data/replicate_otto_fast_pipeline_source_data/train_meta_data.csv")
df_test = pd.read_csv("../../allData/submission_phase_data/replicate_otto_fast_pipeline_source_data/test_meta_data.csv")
df = pd.concat([df, df_test]).reset_index(drop = True)
npz = np.load("../../allData/submission_phase_data/replicate_otto_fast_pipeline_source_data/train_core_data.npz")
npz_test = np.load("../../allData/submission_phase_data/replicate_otto_fast_pipeline_source_data/test_core_data.npz")
aids = np.concatenate([npz['aids'], npz_test['aids']])
ts = np.concatenate([npz['ts'], npz_test['ts']])
ops = np.concatenate([npz['ops'], npz_test['ops']])

df["start_idx"] = df['total_action'].cumsum().shift(1).fillna(0).astype(int)
df["end_time"] = ts[df["start_idx"] + df["total_action"] - 1]

CPU times: user 10.1 s, sys: 4.22 s, total: 14.3 s
Wall time: 15.1 s


In [3]:
## sanity check 
print("len(ts):", len(ts))
print("ts[32564]:", ts[32564])
print("len(aids):", len(aids))
print("aids[9343654]:", aids[9343654])
print("len(ops):", len(ops))
print("ops[797456]: ", ops[797456])
print("df shape:", df.shape)

len(ts): 223644219
ts[32564]: 1659305046
len(aids): 223644219
aids[9343654]: 838109
len(ops): 223644219
ops[797456]:  0
df shape: (14571582, 6)


## 1. Training -- Derive ItemCF similarity Matrix

### CONSTANTS

In [9]:
## Define constants
PARALLEL = 1024
LOOKBACK_WINDOW = 100   ## only fit the latest LOOKBACK_WINDOW to train the sim matrix
#TOPN = 20
ACTION_WEIGHTS = np.array([1.0, 6.0, 3.0])

#### Section A: Utils Functions 
1. Count Item Total likes: The similary score will be normalized by "Item Total Like Scores". In theory, popular items should have less weight in simiarity score.
2. Trimming function: Helpful managing memoery usage. 
3. Method for normalization: Mostly item total like normalization, and max norm(make all sim score between 0 and 1) of the score. 

In [10]:
# ==================================
# Methods for counting Item Total Likes
# ==================================
@nb.jit(nopython=True)
def getItemTotalLikesNaive(aids, ops, item_total_likes, action_weights):
    """
    Stores the total like score of itemXXX in item_total_likes, based on action_weights parameter. np.array([X, Y, Z])
    """
    for idx, item in enumerate(aids):
        if item not in item_total_likes: 
            item_total_likes[item] = 0
        item_total_likes[item] += action_weights[ops[idx]]   ## TODO: For time decay, consider replace with 1, for iuf keep this. 

# ==================================
# Methods for rank and trim the sim score dict
# ==================================
@nb.jit(nopython = True)
def heap_topk(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure
    """
    dic = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q) for _ in range(len(q))][::-1]
    for i in range(len(res)):
        dic[res[i][1]] = res[i][0]
    
    return dic
   
@nb.jit(nopython = True)
def trim_simMatrix_topk(fullSimMatrix, k = 50):
    """
    trim top k items of each "itemX: {itemY: score1, ...}" pair in fullSimMatrix based on sim scores. 
    """
    for item, item_cnt_dict in fullSimMatrix.items():
        fullSimMatrix[item] = heap_topk(item_cnt_dict, k)

# ==================================
# Methods for score normalization
# ==================================

# @nb.jit(nopython=True)
# def itemTotalLikeNorm(fullSimMatrix, item_total_likes):
#     for aid_1, relations in fullSimMatrix.items():
#         for aid_2, sim_score in relations.items():
#             fullSimMatrix[aid_1][aid_2] = sim_score / (item_total_likes[aid_1] * item_total_likes[aid_2]) ** 0.1  ## TODO: consider 0.1 or other small number
            
@nb.jit(nopython=True)
def maxNormSimMatrix(fullSimMatrix):
    for aid_1, relations in fullSimMatrix.items():
        max_num = -np.inf
        for _, sim_score in relations.items():
            if sim_score > max_num:
                max_num = sim_score
        ## DEGUG use, delete later
        if max_num == 0:
            print(aid_1)
            print(fullSimMatrix[aid_1])
        for aid_2, sim_score in relations.items():
#             if max_num == 0:
#                 max_num += 0.001
            fullSimMatrix[aid_1][aid_2] = sim_score / max_num

#### Section B: Sim Score Computation functions

In [11]:
@nb.jit(nopython=True)
def getSimScoresSingleRow(pairs_this_row, start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode):
    """
    Get the sim scores of items within single session, can be ran in parallel within each batch. 
    """
    max_idx = start_idx + length
    min_idx = max(max_idx - LOOKBACK_WINDOW, start_idx)  
    for i in range(min_idx, max_idx):
        for j in range(i+1, max_idx):
            if ts[j] - ts[i] > 2 * 60 * 60: continue  #TODO: try 2h only
            if aids[i] == aids[j]: continue
            
            if mode == "cosine":
                w_ij = action_weights[ops[j]] 
                w_ji = action_weights[ops[i]] 
            elif mode == "iuf":  ## penalize users that had lots of actions TODO: consider location weight
                
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                w_ij = action_weights[ops[j]] * time_gap_weight * loc_weight / math.log1p(length)
                w_ji = action_weights[ops[i]] * time_gap_weight * loc_weight / math.log1p(length)
            elif mode == "time_decay":
                ## calculate some time weights of each item, more weights are given when ts is later. #TODO: try adding (i-j) location weight, exponential weight, 0.5 ** (abs(i-j + 1)), 
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                #time_i = 1 + 0.1 ** ((1662328791-ts[i])/(1662328791-1659304800)) #1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800) #  #(1 - 0.8 *(TEST_END_TS - ts[i]) / TIME_SPAN) ** 0.5 # 0.2~1 #   ## time decay weight for item i 
                #time_j = 1 + 0.1 ** ((1662328791-ts[j])/(1662328791-1659304800))  # 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) # #  #(1 - 0.8 *(TEST_END_TS - ts[j]) / TIME_SPAN) ** 0.5   # 
                time_i = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[i])/(1662328791-1659304800)) - 0.6  )))
                time_j = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[j])/(1662328791-1659304800)) - 0.6  )))
                
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                
                w_ij = action_weights[ops[j]] * loc_weight * time_gap_weight * time_i / math.log1p(length)
                w_ji = action_weights[ops[i]] * loc_weight * time_gap_weight * time_j / math.log1p(length)
                
            pairs_this_row[(aids[i], aids[j])] = w_ij / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1
            pairs_this_row[(aids[j], aids[i])] = w_ji / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1

@nb.jit(nopython=True, parallel=True, cache=True)
def getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, action_weights, item_total_likes, mode="cosine"):
    nrows = len(rows)
    pairs_this_batch = [{(0, 0): 0.0 for _ in range(0)} for _ in range(nrows)]
    ## get the sim scores of each batch in seperate sub dict in pairs_this_batch
    for row_i in nb.prange(nrows):  ## run each row of the batch in parallel
        _, start_idx, length, start_time = rows[row_i]
        getSimScoresSingleRow(pairs_this_batch[row_i], start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode)
    ## merge pairs_this_batch into the fullSimMatrix
    for row_i in range(nrows):
        for (aid1, aid2), score in pairs_this_batch[row_i].items():
            if aid1 not in fullSimMatrix: 
                fullSimMatrix[aid1] = {0: 0.0 for _ in range(0)}
            if aid2 not in fullSimMatrix[aid1]:
                fullSimMatrix[aid1][aid2] = 0.0
            fullSimMatrix[aid1][aid2] += score


#### Section C: Train the similarity matrices
1. Derive the total like score first
2. Train 2 similarity matrices, one using iuf(Inverse User Frequence), the other using time_decay method. 

In [12]:
%%time
## get the Total Like matrix
item_total_likes = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.float64)

getItemTotalLikesNaive(aids, ops, item_total_likes, ACTION_WEIGHTS)

CPU times: user 27.5 s, sys: 1.3 s, total: 28.8 s
Wall time: 30.2 s


In [13]:
%%time
simMatrices = {}   ## store a few different similarity matrices using different scoring system, for different prediction type
TRIM_CYCLES = 1000   ## trim full sim matrix every XX batches. 
MODES_TO_TRAIN = ["iuf"] #, "time_decay"]

for mode in MODES_TO_TRAIN:
    ## the nested dict to store full sim matrix, {itemX: {itemY: score, itemZ: score, ...}}
    fullSimMatrix = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
    max_idx = len(df)
    batch_idx = 1  ## compute sim matrix for PARALLEL # of rows per batch, have a total of max_idx/PARALLEL batches.
    for idx in tqdm(range(0, max_idx, PARALLEL)):
        rows = df.iloc[idx: min(idx + PARALLEL, max_idx)][['session', 'start_idx', 'total_action', 'session_start_time']].values
        getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, ACTION_WEIGHTS, item_total_likes, mode=mode)
        batch_idx += 1
        if batch_idx % TRIM_CYCLES == 0:
            print("batch_idx: ", batch_idx)
            trim_simMatrix_topk(fullSimMatrix, 100)
            gc.collect()
            # break

    
    ## trim top 50 when the training is complete
    trim_simMatrix_topk(fullSimMatrix, 80)   ## TODO: make this num small enough to reduce time for normalization, consider keeping 100, give more option for selection
    ## max norm of each score
    maxNormSimMatrix(fullSimMatrix)
    
    simMatrices[mode] = fullSimMatrix
    
    del fullSimMatrix
    gc.collect()

  7%|▋         | 998/14231 [01:37<31:40,  6.96it/s] 

batch_idx:  1000


 14%|█▍        | 1999/14231 [04:22<36:11:24, 10.65s/it]

batch_idx:  2000


 21%|██        | 2998/14231 [06:28<32:01,  5.85it/s]   

...

batch_idx:  13000


 98%|█████████▊| 13999/14231 [30:53<14:41,  3.80s/it]  

batch_idx:  14000


100%|██████████| 14231/14231 [30:59<00:00,  7.65it/s]


CPU times: user 55min 10s, sys: 28min 52s, total: 1h 24min 3s
Wall time: 32min 26s


In [21]:
gc.collect()

58

#### Section D: Utils for inference:
1. Select top items to recommend in re-ranking
2. Compute Real time importance of each action (Not in use currently).

In [14]:
@nb.jit(nopython = True)
def heap_topk_return_list(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure, return a list with top "cap" elements with highest score
    """
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q)[1] for _ in range(len(q))][::-1]
    
    return res

### Section F: Saving Feature, Run E or F, depends on mode, this will not only generate result, but the featues associated.
**Feature engineering are made here**. 

##### Utils for features save

In [15]:
@nb.jit(nopython=True)
def update_feature_vec(aid, features_tuple_arr, features_idx_map, new_feat_tuple):
    ## append features
    if aid not in features_idx_map:
        features_tuple_arr.append(new_feat_tuple)
        new_pos = len(features_tuple_arr)-1
        ## save the position in the tuple arr
        features_idx_map[aid] = new_pos
    else: # <is_prev_int, seq_w, time_w, total_ops_w, session_len, # uniuqe aids, CF_score, aid's itemTotalLike >
        # ================== 8 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot8_ref_time = features_tuple_arr[features_idx_map[aid]][8]
        else:
            slot8_ref_time = features_tuple_arr[features_idx_map[aid]][8] + new_feat_tuple[8]
        # ================== 9 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot9_max_sim = 1
        else:
            slot9_max_sim = max(new_feat_tuple[9], features_tuple_arr[features_idx_map[aid]][9])
        # ================== 10 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot10_mean_sim = 1
        else:
            slot10_mean_sim = ((features_tuple_arr[features_idx_map[aid]][10] * (features_tuple_arr[features_idx_map[aid]][8]-1) ) + new_feat_tuple[10] ) / features_tuple_arr[features_idx_map[aid]][8]
        # ================== 14 seq_w ==
        slot14_max_seq_w = max(new_feat_tuple[14], features_tuple_arr[features_idx_map[aid]][14])
        # ================== 15 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot15_mean_seq_w = (features_tuple_arr[features_idx_map[aid]][1] + new_feat_tuple[1]) / (features_tuple_arr[features_idx_map[aid]][11] + new_feat_tuple[11])
        else:
            slot15_mean_seq_w = (features_tuple_arr[features_idx_map[aid]][1] + new_feat_tuple[1]) / (features_tuple_arr[features_idx_map[aid]][8] + new_feat_tuple[8])
        # ================== 16 ==
        slot16_min_seq_w = min(new_feat_tuple[16], features_tuple_arr[features_idx_map[aid]][16])
        # ================== 17 time_w == 
        slot17_max_time_w = max(new_feat_tuple[17], features_tuple_arr[features_idx_map[aid]][17])
        # ================== 18 == 
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot18_mean_time_w = (features_tuple_arr[features_idx_map[aid]][2] + new_feat_tuple[2]) / (features_tuple_arr[features_idx_map[aid]][11] + new_feat_tuple[11])
        else:
            slot18_mean_time_w = (features_tuple_arr[features_idx_map[aid]][2] + new_feat_tuple[2]) / (features_tuple_arr[features_idx_map[aid]][8] + new_feat_tuple[8])
        # ================= 19 ==
        slot19_min_time_w = min(new_feat_tuple[19], features_tuple_arr[features_idx_map[aid]][19])
        # ================= 20 ops_w ==
        slot20_max_ops_w = max(new_feat_tuple[20], features_tuple_arr[features_idx_map[aid]][20])
        # ================= 21 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot21_mean_ops_w = (features_tuple_arr[features_idx_map[aid]][3] + new_feat_tuple[3]) / (features_tuple_arr[features_idx_map[aid]][11] + new_feat_tuple[11])
        else:
            slot21_mean_ops_w = (features_tuple_arr[features_idx_map[aid]][3] + new_feat_tuple[3]) / (features_tuple_arr[features_idx_map[aid]][8] + new_feat_tuple[8])
        # ================= 22 ==
        slot22_min_ops_w = min(new_feat_tuple[22], features_tuple_arr[features_idx_map[aid]][22]) #new_feat_tuple[22] if new_feat_tuple[22] < features_tuple_arr[features_idx_map[aid]][22] else features_tuple_arr[features_idx_map[aid]][22]
        # ================= 28 cf_incre ==
        slot28_max_cf_incre = max(new_feat_tuple[28], features_tuple_arr[features_idx_map[aid]][28])
        # ================= 29 ==
        if features_tuple_arr[features_idx_map[aid]][0]: 
            slot29_mean_cf_incre = new_feat_tuple[6] / (features_tuple_arr[features_idx_map[aid]][11] + new_feat_tuple[11])
        else:
            slot29_mean_cf_incre = new_feat_tuple[6] / (features_tuple_arr[features_idx_map[aid]][8] + new_feat_tuple[8])
        # ================= 30 ==
        slot30_min_cf_incre = min(new_feat_tuple[30], features_tuple_arr[features_idx_map[aid]][30])

        
        features_tuple_arr[features_idx_map[aid]] = (new_feat_tuple[0], 
                                                     features_tuple_arr[features_idx_map[aid]][1] + new_feat_tuple[1], 
                                                     features_tuple_arr[features_idx_map[aid]][2] + new_feat_tuple[2], 
                                                     features_tuple_arr[features_idx_map[aid]][3] + new_feat_tuple[3],
                                                     new_feat_tuple[4],
                                                     new_feat_tuple[5],
                                                     new_feat_tuple[6],
                                                     new_feat_tuple[7],
                                                     slot8_ref_time,
                                                     slot9_max_sim,
                                                     slot10_mean_sim,
                                                     features_tuple_arr[features_idx_map[aid]][11] + new_feat_tuple[11],
                                                     new_feat_tuple[12],
                                                     new_feat_tuple[13],
                                                     slot14_max_seq_w,
                                                     slot15_mean_seq_w,
                                                     slot16_min_seq_w,
                                                     slot17_max_time_w,
                                                     slot18_mean_time_w,
                                                     slot19_min_time_w,
                                                     slot20_max_ops_w,
                                                     slot21_mean_ops_w,
                                                     slot22_min_ops_w,
                                                     features_tuple_arr[features_idx_map[aid]][23] + new_feat_tuple[23], 
                                                     features_tuple_arr[features_idx_map[aid]][24] + new_feat_tuple[24],
                                                     features_tuple_arr[features_idx_map[aid]][25] + new_feat_tuple[25],
                                                     new_feat_tuple[26],
                                                     new_feat_tuple[27],
                                                     slot28_max_cf_incre,
                                                     slot29_mean_cf_incre,
                                                     slot30_min_cf_incre,
                                                     slot14_max_seq_w - slot16_min_seq_w,
                                                     slot17_max_time_w - slot19_min_time_w,
                                                     slot20_max_ops_w - slot22_min_ops_w,
                                                     slot28_max_cf_incre - slot30_min_cf_incre
                                                     )

# <
# slot_0: is_prev_interacted, 
# slot_1: seq_w_total, 
# slot_2: time_w_total, 
# slot_3: ops_w_total: for visited item, ops weight total in this session; for unvisited item, ops weight total of the item referencing this item. 
# slot_4: session_len, 
# slot_5: num uniuqe aids, 
# slot_6: CF_score, 
# slot_7: aid's itemTotalLike: total like score use for normalization.  
# slot_8: reference time by similar matrix(if aid visited, default 100; if aid not visited, 1-19(when all aid only interact once, could grow to very large if a lot of actions on one aid), depending how many aid reference this item)
# slot_9: max_sim_score:  (1 if it's a visited item)
# slot_10: mean_sim_score: (1 if it's a visited item)
# slot_11: num_interact, (0 for unvisited item; count of interaction for visited item)
# slot_12: time_span of the session 
# slot_13: action_recency: time to last action(end time), for unvisited items -> the time to of reference_aid to the last action
# slot_14: seq_w_max: 
# slot_15: seq_w_mean: for visited item -> seq_w_total / num_interact; for unvisited item -> seq_w_total / reference_time
## ========================= round 2 ================
# slot_16: seq_w_min: 
# slot_17: time_w_max:
# slot_18: time_w_mean: similar to slot_15
# slot_19: time_w_min
# slot_20: ops_w_max:
# slot_21: ops_w_mean:
# slot_22: ops_w_min: 
# slot_23: num_clicks: visited item, direct num; unvisited item, take the reference item's num
# slot_24: num_carts:
# slot_25: num_orders:
# slot_26: last_action_type: 0 -> clicks, 1-> carts, 2 -> orders
# slot_27: time_to_now: latest interaction time to now 
# slot_28: cf_increment_max: 
# slot_29: cf_increment_mean:
# slot_30: cf_increment_min:
##  ======================== Derivable ============================
## slot_31: seq_w max_min_gap: slot_14 - slot_16 
## slot_32: time_w_max_min_gap: slot_17 - slot_19
## slot_33: ops_w_max_min_gap: slot_20 - slot_22
## slot_34: cf_incre_max_min_gap: slot_28 - slot_30
# >
FEATURE_TUPLE_TEMPLATE = (bool(0), np.float64(0.0), np.float64(0.0), np.int64(0), np.int64(0), np.int64(0), \
    np.float64(0.0), np.float64(0.0), np.int32(0), np.float32(0.0), np.float64(0.0), np.int32(0), np.float64(0.0), np.float64(0.0), np.float32(0.0), np.float32(0.0),\
        np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0),\
            np.int32(0), np.int32(0), np.int32(0), np.int32(0), np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0), \
                np.float32(0.0), np.float32(0.0), np.float32(0.0), np.float32(0.0))

FEATURE_NAMES = ["prev_int", "seq_w_total", "time_w_total", "ops_w_total", "session_len", "num_uniuqe_aids", "CF_score", "itemTotalLike", "ref_time", "max_sim_score",\
    "mean_sim_score", "num_interact", "time_span", "action_recency", "seq_w_max", "seq_w_mean", "seq_w_min", "time_w_max", "time_w_mean", "time_w_min", \
        "ops_w_max", "ops_w_mean", "ops_w_min", "num_clicks", "num_carts", "num_orders", "last_action_type", "time_to_now", "cf_incre_max", "cf_incre_mean", \
            "cf_incre_min", "seqW_max_min_gap", "timeW_max_min_gap", "opsW_max_min_gap", "cf_incre_max_min_gap"]

##### Main feature save logics

In [16]:
@nb.jit(nopython=True)
def save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, item_total_likes, test_ops_weights):
    NOW_TIME = ts[-1] ## ts of latest avaiable action
    PREV_INTERACT_BONUS = 30
    NEARBY_ACTION_BONUS = 1.5
    
    ending_idx = starting_idx + length 
    end_time = ts[ending_idx - 1]
    time_span = end_time - start_time
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]
    
    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
    
    ## get unique aid of each session 
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    for a in candidates:
        unique_aids[a] = 0
    
    ## Sequence weight to all the candidates, from near to far 
    sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
    
    ## Time weight of all candidates, from near to far
    time_weights = []
    time_lapse = end_time - start_time + 1  ## +1 to avoid zero
    for idx in range(starting_idx, ending_idx):
        if end_time - ts[idx] < 2 * 60 * 60:   ## apply nearby action bonus
            time_weight = (1 + 0.5 ** ((end_time - ts[idx])/time_lapse)) * NEARBY_ACTION_BONUS
        else:
            time_weight = 1 + 0.5 ** ((end_time - ts[idx])/time_lapse)
        time_weights.append(time_weight)
    time_weights = time_weights[::-1]
    
    ## feature vector template: [aid: <is_prev_int, seq_w, time_w, associated_action, session_len,.. >]
    features_tuple_arr = nb.typed.List()
    features_tuple_arr.append(FEATURE_TUPLE_TEMPLATE)
    features_idx_map = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.int64)

    helper_idx = starting_idx
    ## making inference
    if len(unique_aids) >= 20:  
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            ## caculate scores
            cf_incre = seq_w * time_w * test_ops_weights[op]
            potential_to_recommend[aid] += cf_incre #* PREV_INTERACT_BONUS
            ## append features
            update_feature_vec(aid, features_tuple_arr, features_idx_map, \
                (1, seq_w, time_w, test_ops_weights[op], length, len(unique_aids), potential_to_recommend[aid], \
                    item_total_likes[aid], 100, 1, 1, 1, time_span, end_time-ts[helper_idx], seq_w, seq_w, seq_w, \
                        time_w, time_w, time_w, test_ops_weights[op], test_ops_weights[op], test_ops_weights[op], op==0, op==1, op==2, op, NOW_TIME-ts[helper_idx],\
                            cf_incre, cf_incre, cf_incre, 0, 0, 0, 0))
            helper_idx += 1
    else:   ## otherwise, fill the rest with similar items.
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            ## get the scores
            cf_incre = seq_w * time_w * test_ops_weights[op] * PREV_INTERACT_BONUS
            potential_to_recommend[aid] += cf_incre
            ## append features
            update_feature_vec(aid, features_tuple_arr, features_idx_map, \
                (1, seq_w, time_w, test_ops_weights[op], length, len(unique_aids), potential_to_recommend[aid], \
                    item_total_likes[aid], 100, 1, 1, 1, time_span, end_time-ts[helper_idx], seq_w, seq_w, seq_w, \
                        time_w, time_w, time_w, test_ops_weights[op], test_ops_weights[op], test_ops_weights[op], op==0, op==1, op==2, op, NOW_TIME - ts[helper_idx],\
                            cf_incre, cf_incre, cf_incre, 0, 0, 0, 0))
            ## adding the similar items, if full_sim_matrix don't have such record, skip. 
            if aid not in full_sim_matrix:
                continue
            for similar_item in full_sim_matrix[aid]:
                ## if sim_item is in candidates, would be included above anyways, skip 
                if similar_item in candidates:
                    continue
                if similar_item not in potential_to_recommend:
                    potential_to_recommend[similar_item] = 0
                
                cf_incre = seq_w * time_w * test_ops_weights[op] * full_sim_matrix[aid][similar_item]
                potential_to_recommend[similar_item] += cf_incre  ## no PREV_INTERACT_BONUS as expected, replaced with sim_matrix scores
                ## append features
                update_feature_vec(similar_item, features_tuple_arr, features_idx_map, \
                    (0, seq_w, time_w, test_ops_weights[op], length, len(unique_aids), potential_to_recommend[similar_item], \
                        item_total_likes[similar_item], 1, full_sim_matrix[aid][similar_item], full_sim_matrix[aid][similar_item], 0, \
                            time_span, end_time-ts[helper_idx], seq_w, seq_w, seq_w, time_w, time_w, time_w, test_ops_weights[op], test_ops_weights[op], test_ops_weights[op], op==0, op==1, op==2, op,\
                                NOW_TIME-ts[helper_idx], cf_incre, cf_incre, cf_incre, 0, 0, 0, 0))
            helper_idx += 1

    result[session] = np.array(heap_topk_return_list(potential_to_recommend, 100))  ## Take top 100 for validation runs. 
    
    feature_tuples_this_session = []
    for aid in result[session]:
#         features_save[(session, aid)] = features_tuple_arr[features_idx_map[aid]]
        feature_tuples_this_session.append(features_tuple_arr[features_idx_map[aid]])
    
    return feature_tuples_this_session

In [17]:
# def load_gt_tables(type):
#     """ type -> carts / orders """
#     gt_labels = pd.read_json("../../allData/validationData/out_7day_test/test_labels.jsonl", lines=True)
#     gt_labels['aids'] = gt_labels["labels"].apply(lambda x: x.get(type))
#     gt_labels = gt_labels[gt_labels.aids.notnull()]
#     gt_labels = gt_labels.drop("labels", axis = 1)
#     ## ========= special df to identify the unique session id to look at ================
#     valid_gt_sessions = gt_labels.drop("aids", axis = 1) 
#     ## ========================================================================
#     ## keep go on for gt labels processing
#     gt_labels = gt_labels.set_index(['session']).apply(pd.Series.explode).reset_index()
#     gt_labels["gt"] = 1
#     return valid_gt_sessions, gt_labels

"""
This is different from validation phase, as we don't have any gt data, therefore, all data has to be exploded and save
"""
def process_batch_pipeline_subVersion(rawDf):
    """ rawDf -> Df with session, aids(100), feature_tuple """
    ## Directly explode aids and feature_tuple
    final_df = rawDf.set_index(['session']).apply(pd.Series.explode).reset_index()

    ## open up the feature tuple 
    # for slot_id, f_name in enumerate(FEATURE_NAMES):
    #     final_df[f_name] = final_df["feature_tuple"].apply(lambda x: x[slot_id])
    
    ## slow method
    #final_df[FEATURE_NAMES] = pd.DataFrame([*final_df.feature_tuple], final_df.index)

    ## new method
    features = np.vstack(final_df["feature_tuple"].values)
    temp_df = pd.DataFrame(features)
    del features
    temp_df.columns = [f'{feat_name}' for feat_name in FEATURE_NAMES]
    final_df[temp_df.columns] = temp_df
    del temp_df
    final_df = final_df.drop("feature_tuple", axis = 1)

    return final_df

In [18]:
len(df_test)

1671803

#### Save features as batches

In [19]:
%%time
result_iuf_orders = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

features_all_sessions = [] # session, aid, feature tuple
                          ## session, aid, feature tuple
gc.collect()

## Given there are 1671803 sessions in total, we separate them into K batches
K = 12
# batch_size = 1671803 // K  ##  -> 139316 dealing with around 140k sessions per batch
session_per_batch = len(df_test) // K 

row_idx_cutoffs = [(len(df) - len(df_test)) + (PARALLEL * (session_per_batch//PARALLEL) ) * i for i in range(1, K+3)]   ## batch process every 1024 * 136 rows
# 145 -> 148644 // 1024  -> batch_size // PARALLEL

feature_batch_id = 0

print("feature store starts:")


for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
#     save_features_parallel(rows, aids, ops, ts, result_iuf_orders, simMatrices["iuf"], np.array([2.0, 8.0, 6.0]), orders_features_save)  
    ## run things in parallel
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        features_tuples_this_session = save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result_iuf_orders, simMatrices["iuf"], item_total_likes, np.array([2.0, 6.0, 6.0]))
        features_all_sessions.append(features_tuples_this_session)
    
    if (start_row in row_idx_cutoffs) or (end_row == len(df)):
        ## save batch result
        rawDf = pd.DataFrame({"session": result_iuf_orders.keys(), "aids": result_iuf_orders.values(), "feature_tuple": features_all_sessions})
        batch_result = process_batch_pipeline_subVersion(rawDf)
        batch_result.to_parquet(f"../../allData/submission_phase_data/features_kaggle_eval_set/V_test/batch_result_{feature_batch_id}.parquet")
        ## clean the memory for next batch
        del batch_result, rawDf, features_all_sessions, result_iuf_orders
        gc.collect()
        ## progress update
        print(f"feature_batch_{feature_batch_id} completes saving.")
        feature_batch_id += 1
        ## initiate the struct for new batch again
        result_iuf_orders = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.types.int64[:])
        features_all_sessions = []
        

  0%|          | 0/1633 [00:00<?, ?it/s]

feature store starts:


<ipython-input-16-2fe8ac0fe4f8>:51: NumbaTypeSafetyWarning: unsafe cast from Tuple(int64, float64, float64, float64, int64, int64, float64, float64, int64, float64, float64, int64, int64, int64, float64, float64, float64, float64, float64, float64, float64, float64, float64, int64, int64, int64, int32, int64, float64, float64, float64, float64, float64, float64, float64) to Tuple(bool, float64, float64, int64, int64, int64, float64, float64, int32, float32, float64, int32, float64, float64, float32, float32, float32, float32, float32, float32, float32, float32, float32, int32, int32, int32, int32, float32, float32, float32, float32, float32, float32, float32, float32). Precision may be lost.
  update_feature_vec(aid, features_tuple_arr, features_idx_map, \
<ipython-input-16-2fe8ac0fe4f8>:51: NumbaTypeSafetyWarning: unsafe cast from Tuple(int64, float64, float64, float64, int64, int64, float64, float64, int64, int64, int64, int64, int64, int64, float64, float64, float64, float64, float6

feature_batch_0 completes saving.


 17%|█▋        | 273/1633 [10:31<25:06:30, 66.46s/it]

feature_batch_1 completes saving.


 18%|█▊        | 286/1633 [10:40<50:18,  2.24s/it]   


KeyboardInterrupt: 

### save carts features 

In [61]:
%%time
result_iuf_carts = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

features_all_sessions = [] # session, aid, feature tuple
                          ## session, aid, feature tuple
gc.collect()

## Given there are 1671803 sessions in total, we separate them into K batches
K = 12
session_per_batch = len(df_test) // K   # 1671803 // K  ##  -> 139316 dealing with around 140k sessions per batch

row_idx_cutoffs = [(len(df) - len(df_test)) + (PARALLEL * (session_per_batch//PARALLEL) ) * i for i in range(1, K+3)]   ## batch process every 1024 * 136 rows

feature_batch_id = 0

print("feature store starts:")

for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
#     save_features_parallel(rows, aids, ops, ts, result_iuf_orders, simMatrices["iuf"], np.array([2.0, 8.0, 6.0]), orders_features_save)  
    ## run things in parallel
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        features_tuples_this_session = save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result_iuf_carts, simMatrices["iuf"], item_total_likes, np.array([4.0, 2.0, 5.0]))
        features_all_sessions.append(features_tuples_this_session)
    
    if (start_row in row_idx_cutoffs) or (end_row == len(df)):
        ## save batch result
        rawDf = pd.DataFrame({"session": result_iuf_carts.keys(), "aids": result_iuf_carts.values(), "feature_tuple": features_all_sessions})
        batch_result = process_batch_pipeline_subVersion(rawDf)
        batch_result.to_parquet(f"../../allData/submission_phase_data/features_kaggle_eval_set/V1/carts_features/batch_result_{feature_batch_id}.parquet")
        ## clean the memory for next batch
        del batch_result, rawDf, features_all_sessions, result_iuf_carts
        gc.collect()
        ## progress update
        print(f"feature_batch_{feature_batch_id} completes saving.")
        feature_batch_id += 1
        ## initiate the struct for new batch again
        result_iuf_carts = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.types.int64[:])
        features_all_sessions = []
        
    #break
#     gc.collect()


  0%|          | 0/1633 [00:00<?, ?it/s]

feature store starts:


  8%|▊         | 137/1633 [12:05<66:05:38, 159.05s/it]

feature_batch_0 completes saving.


 17%|█▋        | 273/1633 [23:44<65:13:43, 172.66s/it]

...

feature_batch_9 completes saving.


 92%|█████████▏| 1497/1633 [1:57:40<6:04:34, 160.84s/it] 

feature_batch_10 completes saving.


100%|██████████| 1633/1633 [2:09:39<00:00,  4.76s/it]   

feature_batch_11 completes saving.
CPU times: user 33min 23s, sys: 49min 7s, total: 1h 22min 31s
Wall time: 2h 9min 50s
